My Goal
1) During intiail processing, the 'triframe' is generated. load them and concatenate them.
3) Get left choice, reward differences, and length differences
4) scale them
5) run the fucking code

In [1]:
import os
import sys
path1 = r'C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules'
path2 = r'C:\Users\yangsunhwang\Documents\GitHub\DA_maze\Behavior\Modules'
#path3 = r'C:\Users\yangsunhwang\Documents\GitHub\HC_decode_analyses\Modules'
sys.path += [path1,path2]
print(os.getcwd())

C:\Users\yangsunhwang\Documents\GitHub\dLight_HexMaze_Processing


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
plt.style.use('default')
plt.style.use("seaborn-v0_8-poster")
from multi_rat_da import *
from choiceAnalysisFuncs import *
from hexLevelAnalyses import get_sigRats_fromMeanList
from photometryQuantifications import *
from scipy.stats import wilcoxon
%matplotlib qt

# load trial-level dataframe (triframe) from each session and concatenate

In [3]:
seshDict = {'IM-1322': ["11202020", "11222020", "11242020", "11252020", "11282020", "11292020", "11302020"], \
            'IM-1398': ["09012021", "09032021", "09152021", "09162021", "09202021", "09212021", "09222021", "09232021", "09282021", "09302021", "10052021"], \
            'IM-1434': ["03102022", "03112022", "04072022", "04132022", "04192022"], \
            'IM-1478': ["07132022", "07142022", "07152022", "07162022", "07172022", "07182022", "07192022", "07202022", "07242022", "07252022", "07262022", "07272022"], \
            'IM-1532': ["11212022", "11232022", "11302022", "12012022", "12022022"], \
            # adding my data
            'IM-1638': ["02062024", "03152024", "04012024"]} #, \
#            'IM-1641': ["02062024"]}

photrats = PhotRats(seshDict)
photrats.directory_prefix="W:/Photometry/"
photrats.load_triframes()
photrats.triframe.reset_index(inplace=True) # idk why but copied OG flow
#photrats.add_totTri2fullDf()
print(f'Total number of sessions loaded: {photrats.triframe.session.nunique()}')
print(f'Total number of rows in concatenated DataFrame: {len(photrats.triframe)}')

loaded triframe for IM-1322; 11202020
loaded triframe for IM-1322; 11222020
loaded triframe for IM-1322; 11242020
loaded triframe for IM-1322; 11252020
loaded triframe for IM-1322; 11282020
loaded triframe for IM-1322; 11292020
loaded triframe for IM-1322; 11302020
loaded triframe for IM-1398; 09012021
loaded triframe for IM-1398; 09032021
loaded triframe for IM-1398; 09152021
loaded triframe for IM-1398; 09162021
loaded triframe for IM-1398; 09202021
loaded triframe for IM-1398; 09212021
loaded triframe for IM-1398; 09222021
loaded triframe for IM-1398; 09232021
loaded triframe for IM-1398; 09282021
loaded triframe for IM-1398; 09302021
loaded triframe for IM-1398; 10052021
loaded triframe for IM-1434; 03102022
loaded triframe for IM-1434; 03112022
loaded triframe for IM-1434; 04072022
loaded triframe for IM-1434; 04132022
loaded triframe for IM-1434; 04192022
loaded triframe for IM-1478; 07132022
loaded triframe for IM-1478; 07142022
loaded triframe for IM-1478; 07152022
loaded trifr

# create new dataframes with relevant columns for regression, and plot results

In [4]:
get_portHist(photrats)

get_back2samePortTrials(photrats)

add_rewardLagColumns2triframe(photrats)

add_rewardLagColumns2triframeByPort(photrats,lags=5)

add_samePathLagColumns2triframeByPort(photrats,lags=1)

add_leftLastColumn(photrats)

C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStrings]].fillna(method='ffill')
C:\Users\yangsunhwang\Documents\GitHub\DA_maze\DA\Modules\choiceAnalysisFuncs.py:64: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  photrats.triframe.loc[photrats.triframe.session==sesh,["rt-"+str(l)+"_"+s for s in portStri

# Fig5B
The probability of choosing the left (counterclockwise) of the two available ports after a reward, compared with an omission

### identify whether rat previously took the same or alternative path to the left available port, and if it was rewarded or not

In [5]:
for l in range(1,6):
    photrats.triframe.loc[:,"rt-"+str(l)+"_left"] = photrats.get_left_val(photrats.triframe,factor="rt-"+str(l))
photrats.triframe.loc[:,"samePath_t-1_left"] = photrats.get_left_val(photrats.triframe,factor="samePath_t-1")

### save as csv for regression analysis in R

In [6]:
#photrats.triframe.to_csv(photrats.directory_prefix+"triframe4sameValtLeftChoiceReg.csv")
photrats.triframe.to_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/" + \
                         photrats.triframe['rat'].unique()[0]+ "_" + photrats.triframe['rat'].unique()[-1] + "_triframe4sameValtLeftChoiceReg.csv")

### create a dictionary of reward vs omission choice probabilities when rat previously took same vs alternative path

In [8]:
ratSameValtChoiceProbs,ratNs = create_sameValtChoiceDict(photrats)

### test if same and alt are sig dif from each other

In [9]:
sameVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmSame"]) for rat in ratSameValtChoiceProbs.keys()]
altVals = [np.nanmean(ratSameValtChoiceProbs[rat]["pDiffRwdVsOmAlt"]) for rat in ratSameValtChoiceProbs.keys()]
wilcoxon(sameVals,altVals)

WilcoxonResult(statistic=13.0, pvalue=0.9375)

### visualize individual rat sensitivities to costs and benefits

### visualize result

In [10]:
from photometryQuantifications import plot_sigMarkers

In [11]:
fig = plotChoosePortSameVAlt(photrats,ratSameValtChoiceProbs)
plt.ylim(-.3,.3)
plt.yticks(np.arange(-3,4)/10)

22.0 0.109375
23.0 0.078125


C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1765: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  order = pd.unique(vector)


([<matplotlib.axis.YTick at 0x242c2a0b3d0>,
 [Text(0, -0.3, '−0.3'),
  Text(0, -0.2, '−0.2'),
  Text(0, -0.1, '−0.1'),
  Text(0, 0.0, '0.0'),
  Text(0, 0.1, '0.1'),
  Text(0, 0.2, '0.2'),
  Text(0, 0.3, '0.3')])

# Fig1D
Results of logistic multiple regressions run for each individual rat

In [5]:
photrats.triframe['nom_rwd_a'] = photrats.triframe.pA
photrats.triframe['nom_rwd_b'] = photrats.triframe.pB
photrats.triframe['nom_rwd_c'] = photrats.triframe.pC

In [6]:
photrats.get_log_pchoos_v_costNben()
photrats.regdf = photrats.regdf.loc[photrats.regdf.tri>25,]

In [7]:
photrats.regdf

,rdif,ldif,choose_L,session,rat,tri,block
26,30.0,2,0.0,0,0,26.0,1.0
27,-60.0,0,0.0,0,0,27.0,1.0
28,30.0,-2,0.0,0,0,28.0,1.0
29,30.0,2,1.0,0,0,29.0,1.0
30,30.0,-2,0.0,0,0,30.0,1.0
...,...,...,...,...,...,...,...
8095,-80.0,-2,1.0,42,5,51.0,3.0
8096,40.0,2,0.0,42,5,52.0,3.0
8097,-80.0,-2,1.0,42,5,53.0,3.0
8098,40.0,2,0.0,42,5,54.0,3.0


### get and save dataframe for mixed-effects p(choose L) regression in R

In [8]:
#photrats.regdf.to_csv(photrats.directory_prefix+"\\costVbenDf4reg.csv")

photrats.regdf.to_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/" + \
                      photrats.triframe['rat'].unique()[0]+ "_" + photrats.triframe['rat'].unique()[-1] + "_costVbenDf4reg.csv")
print(f"saved as {photrats.triframe['rat'].unique()[0]}_{photrats.triframe['rat'].unique()[-1]}_costVbenDf4reg.csv")

saved as IM-1322_IM-1638_costVbenDf4reg.csv


### load regression results from R

In [48]:
# regCoefs = pd.read_csv(photrats.directory_prefix+"\\chooseLregCoefsByRat.csv")
# regSum = pd.read_csv(photrats.directory_prefix+"\\chooseLregSummary.csv")
regCoefs = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/" + \
                       photrats.triframe['rat'].unique()[0]+ "_" + photrats.triframe['rat'].unique()[-1] + "_chooseLregCoefsByRat.csv")
regSum = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/" + \
                     photrats.triframe['rat'].unique()[0]+ "_" + photrats.triframe['rat'].unique()[-1] + "_chooseLregSummary.csv")

In [83]:
# regCoefs = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/IM-1322_IM-1641_chooseLregCoefsByRat.csv")
# regSum = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/IM-1322_IM-1641_chooseLregSummary.csv")
regCoefs = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/IM-1322_IM-1532_chooseLregCoefsByRat_5_OG.csv")
regSum = pd.read_csv("C:/Users/yangsunhwang/Documents/Yangsun/R/IM-1322_IM-1532_chooseLregSummary_5_OG.csv")

### import regression results from R

In [60]:
# Fixed effect
regSum

,Unnamed: 0,Estimate,Std. Error,z value,Pr(>|z|)
0,(Intercept),2.480933,0.498128,4.980510,6.341676e-07
1,pRwdScaled,1.706350,0.322582,5.289668,1.225383e-07
2,ldifScaled,-7.338203,0.860159,-8.531214,1.448199e-17


In [84]:
# Random and total effect
pRwdScaled_effect = regSum.loc[regSum['Unnamed: 0'] == 'pRwdScaled', 'Estimate'].values[0]
ldifScaled_effect = regSum.loc[regSum['Unnamed: 0'] == 'ldifScaled', 'Estimate'].values[0]

# Create a new DataFrame for the results
new_regCoefs = regCoefs.copy()

# Add the effects to the corresponding columns
new_regCoefs['pRwdScaled+Fixed'] = new_regCoefs['pRwdScaled'] + pRwdScaled_effect
new_regCoefs['ldifScaled+Fixed'] = new_regCoefs['ldifScaled'] + ldifScaled_effect
new_regCoefs['Experimenter'] = ['Tim' if i < 5 else 'Yang-Sun' for i in range(len(regCoefs))]
# Print the updated DataFrame
new_regCoefs

,Unnamed: 0,(Intercept),pRwdScaled,ldifScaled,pRwdScaled+Fixed,ldifScaled+Fixed,Experimenter
0,0,2.606210,2.142232,-7.985153,3.848582,-15.323356,Tim
1,1,3.168629,1.330800,-8.051717,3.037150,-15.389920,Tim
2,2,1.925824,2.259139,-7.018178,3.965489,-14.356382,Tim
3,3,1.921036,1.595907,-6.330621,3.302257,-13.668824,Tim
4,4,2.727979,1.015374,-7.024480,2.721724,-14.362683,Tim


IM-1322_IM-1641 

Warning message: \
In checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv,  : \
  Model failed to converge with max|grad| = 0.0267558 (tol = 0.002, component 1) 

### plot regression results from R (summarized and individually plotted)

In [12]:
fig = plt.figure(figsize=(3.5,4))
xvals = ["p(reward)","Distance cost"]
sns.barplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='darkgrey',alpha=.5)
sns.stripplot(data=regCoefs.loc[:,["pRwdScaled","ldifScaled"]],color='k',size=8,marker='X',alpha=.9, jitter=True)
plt.axhline(y=0,ls='--',color='k')
plt.xticks([0,1],xvals,fontweight="bold")
plotRegSigLevel(regSum,0,2.0,1)
plotRegSigLevel(regSum,1,2.0,2)
plt.ylim(-9,3)
plt.yticks(np.arange(-7.5, 5, 2.5))
plt.ylabel("Choice ß value",fontsize='xx-large',fontweight='bold')
plt.tight_layout()

C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\categorical.py:166: FutureWarning: Setting a gradient palette using color= is deprecated and will be removed in version 0.13. Set `palette='dark:k'` for same effect.
  warnings.warn(msg, FutureWarning)
C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):


In [43]:
list(regCoefs['(Intercept)'])

[2.63714132129597,
 3.32720855588254,
 1.96764069827065,
 2.0401323716277,
 3.00626086029785,
 3.31219425134059]

### plot regression results from R (compared by experimenter)

In [58]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
# list(regCoefs['(Intercept)'])
# list(regCoefs['pRwdScaled'])
# list(regCoefs['ldifScaled'])
# Sample data setup (as provided in your description)
data = {
    'Intercept': list(regCoefs['(Intercept)']),
    'pRwdScaled': list(regCoefs['pRwdScaled']),
    'ldifScaled': list(regCoefs['ldifScaled'])
}
regCoefs = pd.DataFrame(data)

# Add a new column for the group label
regCoefs['Group'] = ['Tim' if i < 5 else 'Yang-Sun' for i in range(len(regCoefs))]

# Melt the DataFrame to long format
regCoefs_long = regCoefs.melt(id_vars=['Group'], value_vars=['pRwdScaled', 'ldifScaled'], 
                              var_name='Parameter', value_name='Value')

fig = plt.figure(figsize=(3.5, 4))
xvals = ["p(reward)", "Distance cost"]

# Barplot with color by group, separated by Parameter
sns.barplot(data=regCoefs_long, x='Parameter', y='Value', hue='Group', palette={'Tim': 'darkgrey', 'Yang-Sun': 'lightgrey'}, alpha=0.5)

# Stripplot with color by group, separated by Parameter
sns.stripplot(data=regCoefs_long, x='Parameter', y='Value', hue='Group', size=8, marker='X', alpha=0.9, jitter=True, dodge=True)

plt.axhline(y=0, ls='--', color='k')
plt.xticks(np.arange(2), xvals, fontweight="bold")
# Assuming plotRegSigLevel is a function that adds statistical significance levels
# Update these calls if needed or remove if not applicable
# plotRegSigLevel(regSum, 0, 2.0, 1)
# plotRegSigLevel(regSum, 1, 2.0, 2)
plt.ylim(-9, 3)
plt.yticks(np.arange(-7.5, 5, 2.5))
plt.ylabel("Choice ß value", fontsize='xx-large', fontweight='bold')
plt.tight_layout()

# Adjust the legend to handle duplicates and only show group information
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))  # This removes duplicate labels in legend
plt.legend(by_label.values(), by_label.keys(), title='Group')



C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
C:\Users\yangsunhwang\anaconda3\Lib\site-packages\seaborn\_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
